In [19]:
import pandas as pd
import geopandas as gpd
import numpy as np
import datetime
from shapely.geometry import Polygon
from ydata_profiling import ProfileReport
import json


import json

def coord_lister(geom):
    return (json.loads(geom)['features'][0]['geometry']['coordinates'])

def part_of_journey(hour):
  if hour <= datetime.time(6, 30):
      return '1 - Avant 6h30'
  if datetime.time(6, 30) < hour <= datetime.time(9, 30):
      return '2 - AM (6h30 à 9h30)'
  if datetime.time(9, 30) < hour <= datetime.time(15, 30):
      return '3 - Jour (9h30 à 15h30)'
  if datetime.time(15, 30) < hour <= datetime.time(18, 30):
      return '4 - PM (15h30 à 18h30)'
  if datetime.time(18, 30) < hour :
    return '5 - Soir (après 18h30)'
  else:
    return 'ERROR'

def part_of_journey_stm(period):
  if period == 'Avant 6h30':
      return '1 - Avant 6h30'
  if period == 'AM (6h30 à 9h30)':
      return '2 - AM (6h30 à 9h30)'
  if period == 'Jour (9h30 à 15h30)':
      return '3 - Jour (9h30 à 15h30)'
  if period == 'PM (15h30 à 18h30)':
      return '4 - PM (15h30 à 18h30)'
  if period == 'Soir (après 18h30)':
    return '5 - Soir (après 18h30)'
  else:
    return 'ERROR'


def generate_metatdaa(df, decription, name):
    profile = ProfileReport(
        df,
        title="Metadata "+name,
        missing_diagrams=None,
        correlations=None,
        interactions=None,
        duplicates=None,
        vars={
            "cat": {
                "length": True,
                "characters": False,
                "words": False,
            }
            }
        )
    profile.config.variables.descriptions = decription
    profile.to_file("metadata/metadata_"+name+".html")

### Creation du polygon pour la plaza

In [ ]:
coin_rosemont_boyer= [45.53414890262374, -73.59569524098734]
coin_rosemont_st_vallier = [45.53181801496085, -73.59784906811936]
coin_jean_talon_st_vallier = [45.53915366061581, -73.61386735169552]
coin_jean_talon_boyer = [45.54174620305895, -73.61260342943066]

lat_point_list = [45.53414890262374, 45.53181801496085, 45.53915366061581, 45.54174620305895, 45.53414890262374]
lon_point_list = [-73.59569524098734, -73.59784906811936, -73.61386735169552, -73.61260342943066, -73.59569524098734]

polygon_geom = Polygon(zip(lon_point_list, lat_point_list))
crs = 'epsg:4326'
polygon_plaza = gpd.GeoDataFrame( crs=crs, geometry=[polygon_geom])  


# map = folium.Map([45.53619845067009, -73.60313339047141], zoom_start=15, tiles='cartodbpositron')
# folium.GeoJson(polygon_plaza).add_to(m)
# folium.LatLngPopup().add_to(m)
# map

### Importation des données

In [21]:
# COMMUNAUTO
communauto_df = pd.read_excel('data/Communauto/Données brutes Communauto.xlsx')
communauto_gdf = gpd.GeoDataFrame(communauto_df, geometry=gpd.points_from_xy(communauto_df.ArretLongitude, communauto_df.ArretLatitude))
communauto_gdf.crs = 'epsg:4326'

# extract metadata
columns = {column: column for column in communauto_df.columns.to_list()}
name = 'communauto'
generate_metatdaa(communauto_df, columns, name)


# BIXI
bixi_station_df = pd.read_csv('data/Bixi/2021_stations.csv')
bixi_station_gdf = gpd.GeoDataFrame(bixi_station_df, geometry=gpd.points_from_xy(bixi_station_df.longitude, bixi_station_df.latitude))
bixi_station_gdf.crs = 'epsg:4326'

bixi_data_df = pd.read_csv('data/Bixi/2021_donnees_ouvertes.csv')

bixi_merged_df_1 = pd.merge(bixi_data_df,bixi_station_gdf, 
                     left_on='emplacement_pk_start',
                     right_on=bixi_station_gdf['pk'],
                     how='left')

bixi_merged_df_1 = bixi_merged_df_1.fillna(0)
bixi_merged_df = pd.merge(bixi_merged_df_1, bixi_station_gdf,
                     left_on='emplacement_pk_end',
                     right_on=bixi_station_gdf['pk'],
                     how='left',
                      suffixes=('_start', '_end'))
bixi_merged_df.drop(columns=['latitude_start', 'longitude_start','latitude_end', 'longitude_end'], inplace = True)
bixi_merged_gdf = gpd.GeoDataFrame(bixi_merged_df)

# extract metadata
columns_bixi =  {
    "start_date": "Date et heure de début du déplacement au formant AAAA-MM-JJ hh:mm",
    "emplacement_pk_start": "Identifiant de la station du début du trajet",
    "end_date": "Date et heure de la fin du déplacement au formant AAAA-MM-JJ hh:mm",
    "emplacement_pk_end": "Identifiant de la station de fin du trajet",
    "duration_sec": "Durée total du déplacement en secondes",
    "is_member": "Type d'utilisateur. Valeurs possibles : 1 : Abonné du réseau BIXI Montréal,0 : Non-abonné du réseau BIXI Montréal",
    'pk_start': 'id de la station de depart', 
    'name_start': 'nom de la station de depart', 
    'geometry_start': 'geolcalisation de la station de depart', 
    'pk_end':'id de la station de destination', 
    'name_end': 'nom de la station de destination' , 
    'geometry_end': 'geolcalisation de la station de destination' , 
}
name = 'Bixi'
generate_metatdaa(bixi_merged_df, columns_bixi, name)


# display(bixi_merged_gdf)

# STM
stm_df = pd.read_excel('data/STM/Résultat_Projet FabMob Plaza St-H - Liste des arrêts.xlsx', sheet_name='Données')
stm_gdf = gpd.GeoDataFrame(stm_df, geometry=gpd.points_from_xy(stm_df.X, stm_df.Y))
stm_gdf.crs = 'epsg:2950'
stm_gdf = stm_gdf.to_crs(4326)


# extract metadata
columns = {column: column for column in stm_df.columns.to_list()}
name = 'STM'
generate_metatdaa(stm_df, columns, name)


# AMD
amd_df = pd.read_csv('data/AMD/Taux d\'occupation par troncon - demande du 2022-10-27 - Plaza St-Hubert - v1.csv')
from shapely import wkt
amd_df['geometry'] = amd_df['geometry'].apply(wkt.loads)
amd_gdf = gpd.GeoDataFrame(amd_df, geometry=amd_df.geometry)

amd_gdf.crs = 'epsg:4326'

# extract metadata
columns = {column: column for column in amd_df.columns.to_list()}
name = 'AMD'
generate_metatdaa(amd_df, columns, name)


# display(amd_gdf)

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 820.32it/s]



### Selection des données pour l'experimentation
#### Bixi: 
##### - Selection des données du 1er au 31 octobre 2021
##### - Prise en compte des points d'arrivee seulement
##### - Gestion des periodes de la journée du stop

#### Communauto:
##### - Selection des trajets qui font un arret dans la quartier de la plaza: Type d'arret = Arret ou Nom de station = Flex Montréal
##### - prise en compte des dates et du point de géolocalisation de TripEventDate
##### - Gestion des periodes de la journée du stop

### MTL Trajet
##### Les données n'ont pas ete integrées car elles sont aggrégées pôur H2S donc trop large pour notre cas d'usage

### Parco
##### - les taux d'occupation ont ete regroupée en période de la journée

In [ ]:
import json


def row_to_json(cell):
    # display(list(cell.coords))
    # print(cell)
    return np.array(np.array(cell.coords))
    # return cell.to_json()

def route_to_feature(line):
    data = {
        "type": "Feature",
        "geometry": 
            {
            "type": line.type,
            "coordinates":np.array(np.array(line.coords)).tolist()
            }
    }
    return json.dumps(data)

amd_gdf_atraiter = amd_gdf.copy()
cols = ['Rue','Entre la rue',	'Et la rue','Du coté']
amd_gdf_atraiter['nom_station'] = amd_gdf_atraiter['Rue']
amd_gdf_atraiter['Entre la rue'].loc[amd_gdf_atraiter['Entre la rue'].str.contains('Terrain', case=False)] = ''
amd_gdf_atraiter['Et la rue'].loc[amd_gdf_atraiter['Et la rue'].str.contains('Terrain', case=False)] = ''
amd_gdf_atraiter['Du coté'].loc[amd_gdf_atraiter['Du coté'].str.contains('Terrain', case=False)] = ''

amd_gdf_atraiter['nom_station'] = amd_gdf_atraiter[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
amd_gdf_atraiter['1 - Avant 6h30'] = amd_gdf_atraiter[['Hr00', 'Hr01',  'Hr02',  'Hr03',  'Hr04',  'Hr05',  'Hr06',  'Hr07']].mean(axis=1)
amd_gdf_atraiter['2 - AM (6h30 à 9h30)'] = amd_gdf_atraiter[['Hr06', 'Hr07', 'Hr08',  'Hr09',  'Hr10']].mean(axis=1)
amd_gdf_atraiter['3 - Jour (9h30 à 15h30)'] = amd_gdf_atraiter[['Hr09','Hr10', 'Hr11',  'Hr12',  'Hr13',  'Hr14', 'Hr15',  'Hr16']].mean(axis=1)
amd_gdf_atraiter['4 - PM (15h30 à 18h30)'] = amd_gdf_atraiter[['Hr15',  'Hr16', 'Hr17',  'Hr18',  'Hr19']].mean(axis=1)
amd_gdf_atraiter['5 - Soir (après 18h30)'] = amd_gdf_atraiter[['Hr18',  'Hr19', 'Hr20',  'Hr21',  'Hr22',  'Hr23']].mean(axis=1)
amd_gdf_atraiter.drop(columns=['Rue', 'Entre la rue','Et la rue','Du coté','Hr00','Hr01','Hr02','Hr03','Hr04','Hr05','Hr06','Hr07','Hr08','Hr09','Hr10','Hr11','Hr12','Hr13','Hr14','Hr15','Hr16','Hr17','Hr18','Hr19','Hr20','Hr21','Hr22','Hr23'], axis=1, inplace=True)


amd_gdf_atraiter_2 = amd_gdf_atraiter.melt(id_vars=["No_Troncon",'geometry','nom_station'], var_name="periode", value_name="taux_occupation")
amd_gdf_atraiter_2['IsWeekend'] = 'False'
amd_gdf_atraiter_2['mode'] = 'Stationnement'
amd_gdf_atraiter_2['Nb_personnes']= np.NaN
# amd_gdf_atraiter_2['geom_json']= amd_gdf_atraiter_2['geometry'].to_json()
amd_gdf_atraiter_2['geom_json']= amd_gdf_atraiter_2['geometry'].apply(row_to_json)
amd_gdf_atraiter_2['geom_json_2']= amd_gdf_atraiter_2['geometry'].apply(route_to_feature)

# with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth',1000):  # more options can be specified also
#     display(amd_gdf_atraiter_2['geom_json'].head(20))
# amd_gdf_atraiter_2['geom_array'] = amd_gdf_atraiter_2.geom_json.apply(coord_lister)
amd_gdf_atraiter_2 = amd_gdf_atraiter_2.rename(columns={'No_Troncon': ''})
# amd_gdf_atraiter_2.drop(columns=['geom_json'], axis=1, inplace=True)


display(amd_gdf_atraiter_2)




In [ ]:
communauto_gdf['datDateDebutReservation'] = pd.to_datetime(communauto_gdf['datDateDebutReservation'], format='%d/%m/%y %H:%M')
communauto_gdf['datDateFinReservation'] = pd.to_datetime(communauto_gdf['datDateFinReservation'], format='%d/%m/%y %H:%M')
communauto_gdf['TripEventDate'] = pd.to_datetime(communauto_gdf['TripEventDate'], format='%d/%m/%y %H:%M')
bixi_merged_gdf['end_date'] = pd.to_datetime(bixi_merged_gdf['end_date'], format='%Y-%m-%d %H:%M:%S.%f')


bixi_extract = bixi_merged_gdf[(bixi_merged_gdf['end_date']<= '2021-10-31')& (bixi_merged_gdf['end_date']>= '2021-10-01')].copy()
communauto_gdf['periode'] = communauto_gdf.apply(lambda trip: part_of_journey(trip["TripEventDate"].time()),axis=1)
bixi_extract['periode'] = bixi_extract.apply(lambda trip: part_of_journey(trip["end_date"].time()),axis=1)
stm_gdf['Période'] = stm_gdf.apply(lambda trip: part_of_journey_stm(trip["Période"]),axis=1)



bixi_extract.set_geometry("geometry_end", inplace=True)
bixi_extract_2 = bixi_extract.sjoin(polygon_plaza, how="inner", predicate='within')
display(bixi_extract_2)

bixi_data = bixi_extract_2[['periode', 'end_date', 'emplacement_pk_end', 'name_end','is_member', 'geometry_end']].copy()
bixi_data["type d'arrêt"] = 'Fin de trajet'	
bixi_data["mode"] = 'Bixi'	
# bixi_data.drop(columns='duration_sec', inplace=True)
bixi_data = bixi_data.rename({'end_date': 'date_stop', 'emplacement_pk_end': 'id_station', 'name_end': 'nom_station', "Type d'arrêt": "type_arret", "geometry_end": 'geometry'}, axis=1)

communauto_extract= communauto_gdf[(communauto_gdf["Type d'arrêt"]=='Arrêt') | (communauto_gdf["strNomStation"]=='FLEX Montréal')].copy()
communauto_data = communauto_extract[['periode', 'TripEventDate', 'StationNo', 'strNomStation','Type d\'arrêt','geometry']].copy()

communauto_data['is_member'] =1
communauto_data["mode"] = 'Communauto'	
communauto_data.reset_index(inplace=True)
# communauto_data.drop(columns='TripEventDate_idx', inplace=True)
communauto_data = communauto_data.rename({'TripEventDate': 'date_stop', 'StationNo': 'id_station', 'strNomStation': 'nom_station', "Type d'arrêt": "type_arret"}, axis=1)

# rdf = gpd.GeoDataFrame( pd.concat( dataframesList, ignore_index=True) )

data_plaza =gpd.GeoDataFrame( pd.concat([bixi_data, communauto_data], ignore_index=True))
data_plaza['day_of_week'] = data_plaza['date_stop'].dt.day_name()
data_plaza['IsWeekend'] = data_plaza['date_stop'].dt.weekday >= 5

data_plaza['date'] = data_plaza['date_stop'].dt.date
display(data_plaza)
data_plaza_json =data_plaza.copy()
data_plaza_json['date_stop'] = data_plaza_json['date_stop'].astype(str)
data_plaza_json['date'] = data_plaza_json['date'].astype(str)


In [ ]:
data_plaza_json.to_file("test.geojson", driver='GeoJSON')

In [ ]:
data_plaza_agregat= data_plaza.groupby(['date','periode', 'nom_station','mode']).agg({'id_station':'count', 'IsWeekend': 'first', 'geometry': 'first'}).reset_index()
display(data_plaza_agregat)
data_plaza_agregat.to_excel("data.xlsx")

In [ ]:

stm_gdf_ready = stm_gdf[['Période', 'Description de l\'arrêt', 'Descendants', 'geometry']].copy()
stm_gdf_ready['IsWeekend']= 'False'
stm_gdf_ready['mode']= 'Bus'
stm_gdf_ready['Descendants'] = stm_gdf_ready['Descendants']*-1
stm_gdf_ready = stm_gdf_ready.rename({'Période': 'periode','Description de l\'arrêt': 'nom_station','Descendants':'Nb_personnes'}, axis=1)
# display(stm_gdf_ready)

data_plaza_agregat_all = data_plaza_agregat.groupby(['IsWeekend','periode', 'nom_station','mode']).agg({'id_station':'mean', 'geometry': 'first'}).reset_index()
data_plaza_agregat_all = data_plaza_agregat_all.rename({'id_station': 'Nb_personnes'}, axis=1)
data_plaza_agregat_all.to_excel("data_agregat_all.xlsx")

data_plaza_agregat_week = data_plaza_agregat_all[data_plaza_agregat_all['IsWeekend']==False]
# display(data_plaza_agregat_week)

data_plaza_agregat_week_gdf = gpd.GeoDataFrame(data_plaza_agregat_week, geometry=data_plaza_agregat_week['geometry'])
data_plaza_agregat_week_gdf =gpd.GeoDataFrame( pd.concat([data_plaza_agregat_week_gdf, stm_gdf_ready], ignore_index=True))

data_plaza_agregat_week_gdf['lon'] = data_plaza_agregat_week_gdf['geometry'].x
data_plaza_agregat_week_gdf['lat'] = data_plaza_agregat_week_gdf['geometry'].y

data_plaza_agregat_week_gdf = pd.concat([data_plaza_agregat_week_gdf, amd_gdf_atraiter_2])

display(data_plaza_agregat_week_gdf)



In [ ]:
import csv

data_plaza_agregat_week_gdf.to_file("data_plaza_agregat_week.geojson", driver='GeoJSON')
data_plaza_agregat_week_gdf[data_plaza_agregat_week_gdf['mode']=='Stationnement'].to_file("data_plaza_agregat_week_AMD.geojson", driver='GeoJSON')
data_plaza_agregat_week_gdf[data_plaza_agregat_week_gdf['mode']=='Stationnement'].to_csv("data_plaza_agregat_week_AMD.csv")
data_plaza_agregat_week_gdf.to_csv("data_plaza_agregat_week.csv") 
display(data_plaza_agregat_week_gdf)